<a href="https://colab.research.google.com/github/psy-phy-org/NLP/blob/main/hyakunin_isshu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 百人一種 形態素解析
https://rinsaka.com/python/hyaku/02-janome.html

In [ ]:
# janomeのインストール
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 489 kB/s 


In [ ]:
# モジュールのインポート
from janome.tokenizer import Tokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools # 2次元リストを1次元化

In [ ]:
url = "https://github.com/rinsaka/sample-data-sets/blob/master/hyaku-utf-8.csv?raw=true"
# CSV ファイルをデータフレームに読み込む
df = pd.read_csv(url)
# データフレームを表示する
df

,id,uta,kana,kajin
0,1,秋の田の かりほの庵の 苫をあらみ わが衣手は 露にぬれつつ,あきのたの かりほのいほの とまをあらみ わがころもでは つゆにぬれつつ,天智天皇
1,2,春すぎて 夏来にけらし 白妙の 衣ほすてふ 天の香具山,はるすぎて なつきにけらし しろたへの ころもほすてふ あまのかぐやま,持統天皇
2,3,あしびきの 山鳥の尾の しだり尾の ながながし夜を ひとりかも寝む,あしびきの やまどりのをの しだりをの ながながしよを ひとりかもねむ,柿本人麻呂
3,4,田子の浦に うち出でて見れば 白妙の 富士の高嶺に 雪はふりつつ,たごのうらに うちいでてみれば しろたへの ふじのたかねに ゆきはふりつつ,山部赤人
4,5,奥山に もみぢ踏み分け 鳴く鹿の 声聞く時ぞ 秋は悲しき,おくやまに もみぢふみわけ なくしかの こゑきくときぞ あきはかなしき,猿丸大夫
...,...,...,...,...
95,96,花さそふ 嵐の庭の 雪ならで ふりゆくものは わが身なりけり,はなさそふ あらしのにはの ゆきならで ふりゆくものは わがみなりけり,入道前太政大臣
96,97,こぬ人を まつほの浦の 夕なぎに 焼くやもしほの 身もこがれつつ,こぬひとを まつほのうらの ゆふなぎに やくやもしほの みもこがれつつ,権中納言定家
97,98,風そよぐ ならの小川の 夕暮れは みそぎぞ夏の しるしなりける,かぜそよぐ ならのをがはの ゆふぐれは みそぎぞなつの しるしなりける,従二位家隆
98,99,人もをし 人も恨めし あぢきなく 世を思ふゆゑに 物思ふ身は,ひともをし ひともうらめし あぢきなく よをおもふゆゑに ものおもふみは,後鳥羽院


In [ ]:
df[df.index == 0]

,id,uta,kana,kajin
0,1,秋の田の かりほの庵の 苫をあらみ わが衣手は 露にぬれつつ,あきのたの かりほのいほの とまをあらみ わがころもでは つゆにぬれつつ,天智天皇


In [ ]:
df[df.id == 1]

,id,uta,kana,kajin
0,1,秋の田の かりほの庵の 苫をあらみ わが衣手は 露にぬれつつ,あきのたの かりほのいほの とまをあらみ わがころもでは つゆにぬれつつ,天智天皇


In [ ]:
# かな以外の列
df[df.id == 1][['id', 'uta', 'kajin']]

,id,uta,kajin
0,1,秋の田の かりほの庵の 苫をあらみ わが衣手は 露にぬれつつ,天智天皇


In [ ]:
# データフレームから「うた(uta)」を取り出して，形態素解析による分かち書きを行い，索引語頻度を求めてみる
# 「id」列と「uta」列をデータフレームから取り出して，NumPy の2次元配列に格納する
uta = df.loc[:, ['id','uta']].values
uta

array([[1, '秋の田の かりほの庵の 苫をあらみ わが衣手は 露にぬれつつ'],
       [2, '春すぎて 夏来にけらし 白妙の 衣ほすてふ 天の香具山'],
       [3, 'あしびきの 山鳥の尾の しだり尾の ながながし夜を ひとりかも寝む'],
       [4, '田子の浦に うち出でて見れば 白妙の 富士の高嶺に 雪はふりつつ'],
       [5, '奥山に もみぢ踏み分け 鳴く鹿の 声聞く時ぞ 秋は悲しき'],
       [6, 'かささぎの 渡せる橋に おく霜の 白きを見れば 夜ぞふけにける'],
       [7, '天の原 ふりさけ見れば 春日なる 三笠の山に 出でし月かも'],
       [8, 'わが庵は 都のたつみ しかぞすむ 世をうぢ山と 人はいふなり'],
       [9, '花の色は うつりにけりな いたづらに わが身世にふる ながめせしまに'],
       [10, 'これやこの 行くも帰るも 別れては 知るも知らぬも あふ坂の関'],
       [11, 'わたの原 八十島かけて 漕ぎ出でぬと 人には告げよ あまのつり舟'],
       [12, '天つ風 雲のかよひ路 吹きとぢよ をとめの姿 しばしとどめむ'],
       [13, 'つくばねの 峰より落つる みなの川 こひぞつもりて 淵となりぬる'],
       [14, '陸奥の しのぶもぢずり 誰ゆゑに 乱れそめにし われならなくに'],
       [15, '君がため 春の野に出でて 若菜つむ わが衣手に 雪はふりつつ'],
       [16, '立ち別れ いなばの山の 峰に生ふる まつとし聞かば 今帰り来む'],
       [17, 'ちはやぶる 神代もきかず 竜田川 からくれなゐに 水くくるとは'],
       [18, '住の江の 岸による波 よるさへや 夢の通ひ路 人めよくらむ'],
       [19, '難波潟 みじかき葦の ふしの間も あはでこの世を 過ぐしてよとや'],
       [20, 'わびぬれば 今はた同じ 難波なる みをつくしても あはむとぞ思ふ'],
       [21, '今こむと 言ひしばかりに 長月の 有明の月を 待ちいでつるかな'],
      

In [ ]:
# 形態素解析を実行する関数 my_Janome を定義します（この関数は以降では使いません）．
def my_Janome(text):
    t = Tokenizer()
    results = t.tokenize(text)
    for token in results:
        print(token)

In [ ]:
# 先頭の歌を取り出して「uta」だけを表示
uta[0][1]

'秋の田の かりほの庵の 苫をあらみ わが衣手は 露にぬれつつ'

In [ ]:
my_Janome(uta[0][1])

秋	名詞,一般,*,*,*,*,秋,アキ,アキ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
田	名詞,一般,*,*,*,*,田,タ,タ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
 	記号,空白,*,*,*,*, ,*,*
かり	動詞,自立,*,*,一段,連用形,かりる,カリ,カリ
ほ	動詞,自立,*,*,五段・ラ行,体言接続特殊２,ほる,ホ,ホ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
庵	名詞,一般,*,*,*,*,庵,アン,アン
の	助詞,連体化,*,*,*,*,の,ノ,ノ
 	記号,空白,*,*,*,*, ,*,*
苫	名詞,一般,*,*,*,*,苫,トマ,トマ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
あら	動詞,自立,*,*,五段・ラ行,未然形,ある,アラ,アラ
み	動詞,非自立,*,*,一段,連用形,みる,ミ,ミ
 	記号,空白,*,*,*,*, ,*,*
わが	連体詞,*,*,*,*,*,わが,ワガ,ワガ
衣手	名詞,一般,*,*,*,*,衣手,コロモデ,コロモデ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
 	記号,空白,*,*,*,*, ,*,*
露	名詞,固有名詞,地域,国,*,*,露,ロ,ロ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
ぬれ	動詞,自立,*,*,一段,連用形,ぬれる,ヌレ,ヌレ
つつ	助詞,接続助詞,*,*,*,*,つつ,ツツ,ツツ


In [ ]:
# 空白文字列の削除
uta[0][1].replace(' ', '')

'秋の田のかりほの庵の苫をあらみわが衣手は露にぬれつつ'

In [ ]:
# 空白を削除した文字列に対して形態素解析した結果を確認してみます．残念ながら，一部のヨミは正しくありません．
my_Janome(uta[0][1].replace(' ', ''))

秋	名詞,一般,*,*,*,*,秋,アキ,アキ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
田	名詞,一般,*,*,*,*,田,タ,タ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
かり	動詞,自立,*,*,一段,連用形,かりる,カリ,カリ
ほ	動詞,自立,*,*,五段・ラ行,体言接続特殊２,ほる,ホ,ホ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
庵	名詞,一般,*,*,*,*,庵,アン,アン
の	助詞,連体化,*,*,*,*,の,ノ,ノ
苫	名詞,一般,*,*,*,*,苫,トマ,トマ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
あら	動詞,自立,*,*,五段・ラ行,未然形,ある,アラ,アラ
み	動詞,非自立,*,*,一段,連用形,みる,ミ,ミ
わが	連体詞,*,*,*,*,*,わが,ワガ,ワガ
衣手	名詞,一般,*,*,*,*,衣手,コロモデ,コロモデ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
露	名詞,形容動詞語幹,*,*,*,*,露,アラワ,アラワ
に	助詞,副詞化,*,*,*,*,に,ニ,ニ
ぬれ	動詞,自立,*,*,一段,連用形,ぬれる,ヌレ,ヌレ
つつ	助詞,接続助詞,*,*,*,*,つつ,ツツ,ツツ


In [ ]:
# 分かち書きをして，1次元リストにして返す関数 wakati() を定義します．
def wakati(text):
    t = Tokenizer()
    results = t.tokenize(text)
    words = []
    for token in results:
        words.append(token.surface) # 表層形
    return words

In [ ]:
# 今定義した関数を用いて最初の歌を分かち書きしてみます．結果が1次元リストになって帰ってきていることがわかります．
words_0 = wakati(uta[0][1].replace(' ', ''))
print(words_0)

['秋', 'の', '田', 'の', 'かり', 'ほ', 'の', '庵', 'の', '苫', 'を', 'あら', 'み', 'わが', '衣手', 'は', '露', 'に', 'ぬれ', 'つつ']
